### HW2 - Multinomial Logistic Regression & SVMs 

### Training and Evaluations

Connect gdrive for train/test datasets:

In [2]:
from google.colab import drive # import lib
drive.mount("/content/gdrive") # mount gdrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Import libraries:

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import math
from pandas import *
from sklearn.model_selection import StratifiedKFold
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

Read Data:

In [4]:
sTrainAdd = "/content/gdrive/MyDrive/MsC/cmp712/hw2/train.csv" # address of train.csv on gdrive 
sTestAdd = "/content/gdrive/MyDrive/MsC/cmp712/hw2/test.csv" # address of train.csv on gdrive 
data_training = read_csv(sTrainAdd)
data_test = read_csv(sTestAdd)

Preprocess Data:

In [5]:
# Training Data Preprocessing
temp = data_training.iloc[:,:]
nRows, nColumns = temp.shape
sClass = "type"
x_training = DataFrame()
y_training = DataFrame()
for idxColumns in range(nColumns):
  temp1 = data_training.iloc[:, idxColumns]
  if temp1._name==sClass:
    y_training = y_training.append(temp1)
  else:
    x_training = x_training.append(temp1)
x_training = x_training.T
y_training = y_training.T
# Test Data Preprocessing
temp = data_test.iloc[:,:]
nRows, nColumns = temp.shape
sClass = "type"
x_test = DataFrame()
y_test = DataFrame()
for idxColumns in range(nColumns):
  temp1 = data_test.iloc[:, idxColumns]
  if temp1._name==sClass:
    y_test = y_test.append(temp1)
  else:
    x_test = x_test.append(temp1)
x_test = x_test.T
y_test = y_test.T

<ipython-input-5-9ae2246f050c>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_training = x_training.append(temp1)
<ipython-input-5-9ae2246f050c>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_training = x_training.append(temp1)
<ipython-input-5-9ae2246f050c>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_training = x_training.append(temp1)
<ipython-input-5-9ae2246f050c>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_training = x_training.append(temp1)
<ipython-input-5-9ae2246f050c>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


Encode Datasets:

In [6]:
# Fix Length Feature (remove 'm' in the end and convert string to float)
# Training:
temp = data_training.iloc[:,:]
nRows, nColumns = temp.shape
for idx in range(nRows):
  temp = x_training.length[idx]
  if isinstance(temp, str):
    x_training.length[idx] = float(temp[:-1])
  elif math.isnan(temp):
    x_training.length[idx] = 9999
# Test:
temp = data_test.iloc[:,:]
nRows, nColumns = temp.shape
for idx in range(nRows):
  temp = x_test.length[idx]
  if isinstance(temp, str):
    x_test.length[idx] = float(temp[:-1])
  elif math.isnan(temp):
    x_test.length[idx] = 9999

In [7]:
# Preprocess "taxonomy" label for MultiLabelBinarizer
temp = x_training.taxonomy
nRows = temp.shape
nRows = nRows[0]
for sample in range(nRows): 
  x_training.taxonomy[sample] = x_training.taxonomy[sample].split()
temp = x_test.taxonomy
nRows = temp.shape
nRows = nRows[0]
unique_list = []
for sample in range(nRows): 
  x_test.taxonomy[sample] = x_test.taxonomy[sample].split()

In [8]:
# One-hot Encoding for Training Dataset:
le = LabelEncoder() # create label object
mle = MultiLabelBinarizer()
x_training.diet = le.fit_transform(x_training.diet)
x_training.lived_in = le.fit_transform(x_training.lived_in)
x_training.period = le.fit_transform(x_training.period)
x_training.named_by = le.fit_transform(x_training.named_by)
#x_training.taxonomy = le.fit_transform(x_training.taxonomy)
x_training.taxonomy = mle.fit_transform(x_training.taxonomy)
x_training.species = le.fit_transform(x_training.species)
# Limit dataset #
x_training_processed = x_training.drop(columns=['id', 'name', 'link']) # x_, argument used in training
print(x_training_processed)
# One-hot Encoding for Test Dataset:
x_test.diet = le.fit_transform(x_test.diet)
x_test.lived_in = le.fit_transform(x_test.lived_in)
x_test.period = le.fit_transform(x_test.period)
x_test.named_by = le.fit_transform(x_test.named_by)
#x_test.taxonomy = le.fit_transform(x_test.taxonomy)
x_test.taxonomy = mle.fit_transform(x_test.taxonomy)
x_test.species = le.fit_transform(x_test.species)
# Limit dataset, drop id, name and link as they will not be used in training #
x_test_processed = x_test.drop(columns=['id', 'name', 'link']) # x_, argument used in training
print(x_test_processed)

     diet  period  lived_in length  taxonomy  named_by  species
0       0     108        25   9999         0        76       61
1       2     108         1   5.15         0        11       95
2       1      65         5   12.0         0       212      188
3       1      43        24   21.0         0        64      181
4       1      21        25   10.0         0        22       24
..    ...     ...       ...    ...       ...       ...      ...
241     1      11        13   9999         0       186      184
242     1      10         2    3.0         0       161      120
243     0      14         5    0.8         0       203      222
244     0      42         1    7.6         0        14      182
245     3     112        19   12.0         0        70      170

[246 rows x 7 columns]
    diet  period  lived_in length  taxonomy  named_by  species
0      1      18        13    5.0         0        36       19
1      1      41        17   23.0         0         8       51
2      0      24   

In [18]:
# Logistic Regression #
# Training and mean_accuracy #
skf = StratifiedKFold(n_splits=5)  #the number of folds is 5
model = LogisticRegression(solver='lbfgs', max_iter=1000)
accuracy_scores = []
for train_index, test_index in skf.split(x_training_processed, y_training):
  X_train, X_test = x_training_processed.iloc[train_index], x_training_processed.iloc[test_index]
  y_train, y_test = y_training.iloc[train_index], y_training.iloc[test_index]  
  model.fit(X_train, y_train)
  accuracy = model.score(X_test, y_test)
  accuracy_scores.append(accuracy)
# Compute the mean accuracy across all folds
np.savetxt('training_LR.csv', model.predict(x_training_processed), delimiter=',', fmt='%s') # save result to text
np.savetxt('test_LR.csv', model.predict(x_test_processed), delimiter=',', fmt='%s') # save result to text
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel

Logistic Regression - Mean accuracy: 0.5157551020408163


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
# Logistic Regression - F1 Score and Confusion Matrix
f1_scores_LR = []
confusion_matrix_LR = []
# Perform k-fold stratified cross-validation
for train_index, test_index in skf.split(x_training_processed, y_training):
    X_train, X_test = x_training_processed.iloc[train_index], x_training_processed.iloc[test_index]
    y_train, y_test = y_training.iloc[train_index], y_training.iloc[test_index] 
    model.fit(X_train, y_train) # Necessary code to compute the predictions using your classifier.
    y_pred = model.predict(X_test)
    # Compute the weighted-average F1-score for this fold
    fold_f1_score = f1_score(y_test, y_pred, average='weighted')
    f1_scores_LR.append(fold_f1_score)
    fold_confusion_matrix = confusion_matrix(y_test, y_pred)
    confusion_matrix_LR.append(fold_confusion_matrix)
# Calculate the mean F1-score across all folds
mean_weighted_f1_score = np.mean(f1_scores_LR)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel

Logistic Regression - Mean weighted-average F1-score across 5 folds: 0.5050917807653164


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
print("Logistic Regression - Mean accuracy:", mean_accuracy)
print("Logistic Regression - Mean weighted-average F1-score across", 5, "folds:", mean_weighted_f1_score)
print("Logistic Regression - Confusion Matrix", confusion_matrix_LR)

Logistic Regression - Mean weighted-average F1-score across 5 folds: 0.5050917807653164
Logistic Regression - Confusion Matrix [array([[1, 0, 2, 0, 2, 0],
       [0, 2, 3, 0, 0, 0],
       [1, 0, 6, 0, 2, 0],
       [0, 0, 0, 7, 1, 2],
       [0, 0, 1, 1, 9, 0],
       [1, 1, 0, 1, 0, 7]]), array([[1, 0, 3, 0, 1, 0],
       [0, 3, 1, 0, 0, 0],
       [2, 1, 4, 0, 2, 0],
       [0, 0, 1, 5, 1, 3],
       [2, 0, 2, 1, 3, 3],
       [0, 0, 0, 6, 1, 3]]), array([[1, 0, 2, 0, 2, 0],
       [0, 4, 0, 0, 0, 0],
       [1, 1, 7, 0, 0, 0],
       [0, 0, 0, 7, 1, 2],
       [0, 0, 0, 1, 6, 4],
       [0, 0, 0, 5, 1, 4]]), array([[1, 0, 1, 0, 4, 0],
       [0, 3, 1, 0, 0, 0],
       [0, 1, 6, 0, 3, 0],
       [0, 0, 0, 2, 2, 5],
       [0, 0, 0, 1, 9, 1],
       [0, 0, 0, 6, 2, 1]]), array([[1, 0, 1, 0, 3, 0],
       [0, 4, 1, 0, 0, 0],
       [2, 1, 6, 0, 1, 0],
       [0, 0, 0, 5, 2, 2],
       [1, 0, 1, 1, 6, 2],
       [0, 0, 0, 5, 1, 3]])]


SVM:

In [18]:
# SVM #
# Training and mean_accuracy #
x_training_processed_SVM = x_training_processed
x_test_processed_SVM = x_test_processed
skf = StratifiedKFold(n_splits=3, shuffle=True)  # the number of folds is 3
model_SVM = SVC(kernel = 'linear', decision_function_shape='ovo')
accuracy_scores = []
for train_index, test_index in skf.split(x_training_processed_SVM, y_training):
  X_train, X_test = x_training_processed_SVM.iloc[train_index], x_training_processed_SVM.iloc[test_index]
  y_train, y_test = y_training.iloc[train_index], y_training.iloc[test_index]
  model_SVM.fit(X_train, y_train)
  y_pred = model_SVM.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  accuracy_scores.append(accuracy)
# Compute the mean accuracy across all folds
np.savetxt('training_SVM.csv', model_SVM.predict(x_training_processed_SVM), delimiter=',', fmt='%s') # save result to text
np.savetxt('test_SVM.csv', model_SVM.predict(x_test_processed_SVM), delimiter=',', fmt='%s') # save result to text
mean_accuracy_SVM = sum(accuracy_scores) / len(accuracy_scores)
print("SVM - Mean accuracy:", mean_accuracy_SVM)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVM - Mean accuracy: 0.4268292682926829


In [19]:
# Calculate SVM F1 score and Confusion Matrix
f1_scores_SVM = []
confusion_matrix_SVM = []
# Perform k-fold stratified cross-validation
for train_index, test_index in skf.split(x_training_processed_SVM, y_training):
    X_train, X_test = x_training_processed_SVM.iloc[train_index], x_training_processed_SVM.iloc[test_index]
    y_train, y_test = y_training.iloc[train_index], y_training.iloc[test_index] 
    model_SVM.fit(X_train, y_train) # Necessary code to compute the predictions using your classifier.
    y_pred = model_SVM.predict(X_test)
    # Compute the weighted-average F1-score for this fold
    fold_f1_score_svm = f1_score(y_test, y_pred, average='weighted')
    f1_scores_SVM.append(fold_f1_score_svm)
    fold_confusion_matrix_SVM = confusion_matrix(y_test, y_pred)
    confusion_matrix_SVM.append(fold_confusion_matrix_SVM)
# Calculate the mean F1-score across all folds
mean_weighted_f1_score_SVM = np.mean(f1_scores_SVM)
print("SVM - Mean weighted-average F1-score across", 5, "folds:", mean_weighted_f1_score_SVM)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVM - Mean weighted-average F1-score across 5 folds: 0.4013251078409716


In [20]:
print("SVM - Mean accuracy:", mean_accuracy_SVM)
print("SVM - Mean weighted-average F1-score across", 5, "folds:", mean_weighted_f1_score_SVM)
print("Logistic Regression - Confusion Matrix", confusion_matrix_SVM)

SVM - Mean accuracy: 0.4268292682926829
SVM - Mean weighted-average F1-score across 5 folds: 0.4013251078409716
Logistic Regression - Confusion Matrix [array([[ 1,  0,  7,  0,  0,  1],
       [ 0,  0,  4,  0,  1,  2],
       [ 0,  0, 13,  0,  1,  1],
       [ 0,  0,  1, 13,  2,  0],
       [ 0,  0,  9,  0,  9,  1],
       [ 0,  0,  2,  1,  0, 13]]), array([[ 0,  0,  8,  0,  1,  0],
       [ 0,  0,  5,  0,  2,  0],
       [ 0,  0, 13,  0,  3,  0],
       [ 1,  0,  1, 11,  2,  1],
       [ 0,  0, 11,  0,  7,  0],
       [ 0,  0,  1, 11,  4,  0]]), array([[ 0,  1,  5,  0,  2,  0],
       [ 0,  0,  4,  0,  4,  0],
       [ 0,  0, 10,  0,  6,  0],
       [ 0,  2,  0,  6,  1,  7],
       [ 0,  1,  5,  0, 12,  0],
       [ 0,  2,  0,  5,  5,  4]])]
